In [104]:
"""
A Cellular automaton simulation of coronavirus cov-19
Created in Apr 2020
"""
import sys
import numpy as np
from numpy import array
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from scipy import signal
import matplotlib._png as png
from skimage.transform import downscale_local_mean

%matplotlib inline
np.set_printoptions(threshold=sys.maxsize)



In [105]:
class Cov():
    """
    this is the model for coronavirus cov-19
    """
    
    
    def __init__(self, initial, bi_initial, ini_edge):

        """
        modify your parameters over here:        
        m, c, h, s, m_control, c_control, k_control...
        """
        # initial spreading coefficient
        self.M_INI = 0.3
        # travel parameter
        self.C = 0.001
        # percentage to hospital
        self.H = 0.08
        # percentage to selfheal
        self.S = 0.87
        # governmental controls
        # self.m_contr = 0.3
        # self.C_CONTR = 0.02
        self.K_CONTR = 0
        # time period of latency(days)
        self.TIME_H = 10
        # time period of selfheal(days)
        self.TIME_S = 28
        # maximum population of a cell
        self.MAX = 10000
        
        # initialize the map
        self.state = initial
        self.bi_cmap = bi_initial
        self.edge = ini_edge
        self.m = np.full_like(self.state, self.M_INI)
        # initialize the sum of specific population
        self.sum_h = np.full_like(self.state, 0)
        self.sum_s = np.full_like(self.state, 0)
        # set up neighbor convolution kernel
        """
        convolution kernel: 
        assume up, down, left and right have more weights than the other four directions
        0.5 1.0 0.5
        1.0 0.0 1.0
        0.5 1.0 0.5
        """
        self.ker = np.array([[0.5,1,0.5],[1,0,1],[0.5,1,0.5]])
        # self.ker_sum = np.sum(self.ker)
        
        
    def count_neighb(self): 
        """
        use 2D convolution to calculate neighbor counts, should be integers between 0 and 8
        """
        neigh_count = signal.convolve2d(self.bi_cmap, self.ker, 'same', boundary = 'fill')
        return neigh_count
        
        
    def count_neighb_popu(self, state):
        """
        use 2D convolution to calculate neighbor populations
        """
        popu_count = signal.convolve2d(state, self.ker, 'same', boundary = 'fill')
        return popu_count
    
    
    def upadate_m(self):
        """
        update m at each step according to the rules of immunity
        """
        self.m = self.m * (1 - (self.state+self.sum_s+self.sum_h)/self.MAX)
        # print(self.m)
        
        
    def spread(self):
        """
        natural spreading according to the R0 rate and short-distance migration
        """
        neighb = self.count_neighb_popu(self.state)
        num = (self.state + (neighb-self.count_neighb()*self.state)*self.C) * (1 + self.m)
        return num
    
    
    def hospital(self):
        """
        population isolated by hospitals
        """
        preneighb = self.count_neighb_popu(self.prestate)
        redu_h = (self.prestate * self.pre_m * self.H * (1-self.C)**self.TIME_H
            + preneighb * self.pre_m * self.H * self.C * ((1-(1-self.C)**self.TIME_H)/(1-(1-self.C))))
        self.sum_h += redu_h
        return redu_h
    
    
    def selfheal(self):
        """
        population that heal themselves outside of hospitals
        """
        preneighb2 = self.count_neighb_popu(self.prestate2)
        redu_s = (self.prestate2 * self.pre_m2 * self.S * (1-self.C)**self.TIME_S
            + preneighb2 * self.pre_m2 * self.S * self.C * ((1-(1-self.C)**self.TIME_S)/(1-(1-self.C))))
        self.sum_s += redu_s
        # print(redu_s[5][5])
        return redu_s
        

    def update_early(self):
        """
        early stage, all patients are in the incubation period 
        """
        self.upadate_m()
        self.state = self.spread()
        self.state[self.edge] = 0

        
    def update_found(self):
        """
        after a period of 10 days
        first case of being tested positive(hospital isolation) appears
        """ 
        self.upadate_m()
        self.state = self.spread() - self.hospital()
        self.state[self.edge] = 0
    
    
    def update_selfheal(self):
        """
        after a period of 25 days
        first case of selfheal appears
        """ 
        self.upadate_m()
        self.state = self.spread() - self.hospital() - self.selfheal()  
        self.state[self.edge] = 0
       
    
    def update_control(self):
        """
        introduce governmental control
        m_control, c_control, K_CONTR
        """
        # self.m = self.m_contr
        # self.C = self.C_CONTR
        self.upadate_m()
        state_bef_contr = self.spread() - self.hospital() - self.selfheal()
        self.state = state_bef_contr * (1 - self.K_CONTR)
        self.state[self.edge] = 0
    
    
    def simulate(self, num_con, num_gen):
        """     
        run the simulation by time steps
        """
        fig = plt.figure(figsize = (10,10))
        plt.axis("off")
        states = []
        ms =[]
        ims = []
        cmax = np.log(self.MAX+1)
        cnorm = mpl.colors.Normalize(vmin=0.,vmax=cmax)
        
        for i in range(self.TIME_H):
            self.state[np.where(self.state < 0)] = 0
            if np.any(np.log(self.state+1) > cmax*0.99):
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            im = plt.imshow(np.log(self.state+1), cmap='Reds', norm = cnorm, animated = True)
            ims.append([im])
            self.update_early()
            
        for i in range(self.TIME_H, self.TIME_S):
            self.state[np.where(self.state < 0)] = 0
            if np.any(np.log(self.state+1) > cmax*0.99):
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            im = plt.imshow(np.log(self.state+1), cmap='Reds', norm = cnorm, animated = True)
            ims.append([im])
            self.update_found()
            
        for i in range(self.TIME_S, num_con):
            self.state[np.where(self.state < 0)] = 0
            if np.any(np.log(self.state+1) > cmax*0.99):
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            # 25 days before
            self.prestate2 = states[i-self.TIME_S]
            self.pre_m2 = ms[i-self.TIME_S]
            im = plt.imshow(np.log(self.state+1), cmap='Reds', norm = cnorm, animated = True)
            ims.append([im])
            self.update_selfheal()
            
        for i in range(num_con, num_gen):
            self.state[np.where(self.state < 0)] = 0
            if np.any(np.log(self.state+1) > cmax*0.99):
                break
            # print("{}:{}".format(i, np.sum(self.state)))
            # print("{}:{}".format(i, self.state[5][5]))
            states.append(self.state)
            ms.append(self.m)
            # 10 days before
            self.prestate = states[i-self.TIME_H]
            self.pre_m = ms[i-self.TIME_H]
            # 25 days before
            self.prestate2 = states[i-self.TIME_S] 
            self.pre_m2 = ms[i-self.TIME_S]
            im = plt.imshow(np.log(self.state+1), cmap='Reds', norm = cnorm, animated = True)
            ims.append([im])
            self.update_control()
            
        # np.savetxt("state40",states[40])    
        anim = animation.ArtistAnimation(
            fig, ims, interval=500, repeat=False, blit=True
        )
        plt.close()
        return anim
    
    

In [110]:
pi = png.read_png_int('england.png')
pi = downscale_local_mean(pi, (20, 20, 1))
print(pi)
# print(pi.dtype)
print(pi.shape)
np.unique(pi.reshape(-1, pi.shape[2]), axis=0)

a = pi.shape[0]
b = pi.shape[1]
cmap = np.zeros((a,b))
bi_cmap = np.zeros((a,b))
for i in range(a):
    for j in range(b):
        if pi[i][j][0] == 255 and pi[i][j][1] == 255 and pi[i][j][2] == 255:
            cmap[i][j] = -1
        else:
            cmap[i][j] = pi[i][j][0]/10
            bi_cmap[i][j] = 1   
    
    

[[[2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.550000e+02]
  [2.550000e+02 2.550000e+02 2.550000e+02 2.5500

In [111]:
init = cmap
ocean = np.where(init == -1)
print(ocean)
init[ocean] = 0
Simulation = Cov(init, bi_cmap, ocean)
anim = Simulation.simulate(100, 100)
#anim.save("simulation.mp4")
HTML(anim.to_html5_video())
#anim.save('animation.gif', writer='imagemagick', fps=10)

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  